<a href="https://colab.research.google.com/github/AnastasiaDMW/ML-Series-1/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Модель 1

### Задача: Сделать фильтр комментариев, после анализа всех комментариев выгружать анализ положительных и отрицательных на экран и все комментарии в файл txt. (Hugging Face)

In [1]:
from transformers import pipeline

model_name = "blanchefort/rubert-base-cased-sentiment-rurewiews"
classifier = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name)

def analyze_comments(comments):
  positive_count = 0
  negative_count = 0
  positive_comments = []
  negative_comments = []

  for comment in comments:
    sentiment = classifier(comment)[0]
    if sentiment['label'] == 'NEGATIVE':
      negative_count += 1
      negative_comments.append(f"Негативный комментарий: {comment}")
    else:
      positive_count += 1
      positive_comments.append(f"Позитивный комментарий: {comment}")

  total = positive_count + negative_count
  if total > 0:
    pos_percent = round((positive_count / total) * 100, 2)
    neg_percent = round((negative_count / total) * 100, 2)
    print("Результаты анализа:")
    print(f"Позитивных: {positive_count} ({pos_percent}%)")
    print(f"Негативных: {negative_count} ({neg_percent}%)")

    write_to_file("comments_result.txt", positive_comments, negative_comments)
  else:
    print("Нет комментариев для анализа")



def write_to_file(path, negative_comments, positive_comments):
  with open(path, "w", encoding="utf-8") as f:
    all_comments = positive_comments + negative_comments
    f.write("\n".join(all_comments))

  print(f"\nВсе комментарии сохранены в файл {path}")


comments = [
    "Это было очень интересно!",
    "Все супер, мне понравилось",
    "Это было ужасно, не снимай больше видео",
    "Все заработало, спасибо, ты лучший",
    "Чтоб ты сдох"
]
analyze_comments(comments)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/950 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/495 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


Результаты анализа:
Позитивных: 3 (60.0%)
Негативных: 2 (40.0%)

Все комментарии сохранены в файл comments_result.txt


## Модель 2

### Задача: Сделать возможность подсказок вариантов текста, если текст пользователя не вместился в нужное кол-во сиволов при введении в текстовое поле (русской модельке тяжело это делать) (PyTorch)

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "sarahai/ruT5-base-summarizer"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

device = 0 if torch.cuda.is_available() else -1

def summarize_to_limit_chars(text, max_chars=50, num_options=3):
    if len(text) <= max_chars:
        print(f"Текст короче {max_chars} символов:")
        return [text]

    print(f"Текст длиннее {max_chars} символов, генерируем варианты...")
    options = []

    for i in range(num_options):
        inputs = tokenizer(text, return_tensors="pt").to(model.device)

        outputs = model.generate(
            **inputs,
            max_length=20,
            min_length=5,
            do_sample=True,
            top_k=50,
            top_p=0.95,
        )

        summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        summary = summary[:max_chars].strip()
        options.append(summary)

    print("\nСгенерированные варианты:")
    for idx, option in enumerate(options, 1):
        print(f"{idx}. {option} ({len(option)} символов)")

    return options


text = """
Сегодня мне привезли не тот заказ, стаканчики помятые, бургер помят, соусы помяты,
салфеток нет, я очень зол на всю эту ситуацию, вы обязаны возместить мне нанесенный ущерб.
"""

options = summarize_to_limit_chars(text, max_chars=60, num_options=3)

choice = int(input("\nВыберите лучший вариант (1-3): "))
print("Ваш выбор:", options[choice-1])

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Текст длиннее 60 символов, генерируем варианты...

Сгенерированные варианты:
1. Заказ был не того заказа, соусы помят, соусы помят, соус (56 символов)
2. Банкет не тот, бургер помят, соусы помяты, бургер (49 символов)
3. В ходе кулинарной конференции стало известно о том, что мне (59 символов)

Выберите лучший вариант (1-3): 1
Ваш выбор: Заказ был не того заказа, соусы помят, соусы помят, соус


## Модель 3

### Задача: Написать код, который распознает аудио и выдает команды в текстовом виде, команды можно использовать для какого-нибудь умного дома. (Hugging Face)

In [3]:
from google.colab import drive
from transformers import pipeline

drive.mount('/content/drive')
model_name = "openai/whisper-small"
asr = pipeline("automatic-speech-recognition", model=model_name)

commands = {
    "включи свет": lambda: print("Свет включён"),
    "выключи свет": lambda: print("Свет выключен"),
    "включи музыку": lambda: print("Музыка играет"),
    "выключи музыку": lambda: print("Музыка остановлена"),
    "открой дверь": lambda: print("Дверь открыта"),
    "закрой дверь": lambda: print("Дверь закрыта"),
    "установи температуру на": lambda temp: print(f"Температура установлена на {temp}°C"),
    "включи увлажнитель воздуха": lambda: print("Увлажнитель воздуха включён"),
    "выключи увлажнитель воздуха": lambda: print("Увлажнитель воздуха выключен"),
}

def handle_command(text):
    text_lower = text.lower()

    for cmd, action in commands.items():
        if cmd in text_lower:
            if "температуру" in cmd:
                import re
                match = re.search(r"\d+", text_lower)
                if match:
                    action(match.group())
                else:
                    print("Не указана температура")
            else:
                action()
            return

    print("Команда не распознана")

def generate_text_from_audio(audio: str):
  recognized_text = asr(audio)["text"]
  print(f"Распознанный текст: {recognized_text}")

  handle_command(recognized_text)

file_path = '/content/drive/My Drive/wav_files/TempOn21.wav'
generate_text_from_audio(file_path)

Mounted at /content/drive


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

preprocessor_config.json: 0.00B [00:00, ?B/s]

Device set to use cpu
`return_token_timestamps` is deprecated for WhisperFeatureExtractor and will be removed in Transformers v5. Use `return_attention_mask` instead, as the number of frames can be inferred from it.
Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


Распознанный текст:  Установи температуру на 21 градусов.
Температура установлена на 21°C


# Модель 4

## Задача: Распознавание по изображению (PyTorch)

In [4]:
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO

model = models.resnet50(pretrained=True)
model.eval()

imagenet_classes = requests.get(
    "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
).text.split("\n")

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

def classify_image(image_path_or_url):
    if image_path_or_url.startswith("http"):
        response = requests.get(image_path_or_url)
        img = Image.open(BytesIO(response.content)).convert("RGB")
    else:
        img = Image.open(image_path_or_url).convert("RGB")

    input_tensor = preprocess(img)
    input_batch = input_tensor.unsqueeze(0)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    input_batch = input_batch.to(device)

    with torch.no_grad():
        output = model(input_batch)

    _, predicted_idx = torch.max(output, 1)
    predicted_label = imagenet_classes[predicted_idx.item()]

    return predicted_label

image_url = "https://woodville.ru/photo/derevyannyy_stol_adelik_160_220_kh90kh76_dub/derevyannyy_stol_adelik_160_220_kh90kh76_dub_575481_1000_700_1_0_1022537.jpg"
predicted = classify_image(image_url)
print(f"Предсказанный объект на изображении: {predicted}")

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 151MB/s]


Предсказанный объект на изображении: dining table


# Модель 5

## Задача: Cделать так, чтобы по отрывку видео можно было определить происходящее. (PyTorch)

In [8]:
!pip install av

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 22.0 MB/s eta 0:00:00


In [9]:
import torch
import torch.nn.functional as F
from torchvision import models
from torchvision.transforms.functional import resize
import numpy as np
import av
from google.colab import drive
from torchvision.models.video import MC3_18_Weights

drive.mount('/content/drive')

device = "cuda" if torch.cuda.is_available() else "cpu"

model = models.video.mc3_18(weights=models.video.MC3_18_Weights.KINETICS400_V1)
model.eval()
model.to(device)

weights = MC3_18_Weights.KINETICS400_V1
kinetics_classes = weights.meta["categories"]
print(len(kinetics_classes))
print(kinetics_classes[:10])

def read_video_frames(video_path, num_frames=16):
    """Считываем равномерно num_frames кадров из видео с помощью PyAV."""
    container = av.open(video_path)
    video_stream = container.streams.video[0]
    total_frames = video_stream.frames
    frame_indices = torch.linspace(0, total_frames-1, steps=num_frames).long()
    frames = []

    for i, frame in enumerate(container.decode(video=0)):
        if i in frame_indices:
            img = frame.to_image()
            img = np.array(img)
            img = torch.tensor(img).permute(2,0,1).float() / 255.0
            img = resize(img, [112,112])
            frames.append(img)
        if len(frames) == num_frames:
            break
    container.close()

    video_tensor = torch.stack(frames).unsqueeze(0)
    video_tensor = video_tensor.permute(0,2,1,3,4).to(device)
    return video_tensor

def classify_scene_mc3(video_path, num_frames=16, topk=3):
    video_tensor = read_video_frames(video_path, num_frames)
    with torch.no_grad():
        outputs = model(video_tensor)
        probs = F.softmax(outputs, dim=1)
        top_prob, top_idx = probs.topk(topk)
        labels = [kinetics_classes[i] if i < len(kinetics_classes) else "Unknown"
                  for i in top_idx[0].tolist()]
        confidences = top_prob[0].tolist()
    return list(zip(labels, confidences))

video_file = "/content/drive/My Drive/video_files/gordostb.mp4"
top3 = classify_scene_mc3(video_file)

print("Топ-3 жанра с вероятностями:")
for label, prob in top3:
    print(f"{label}: {prob:.2f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Downloading: "https://download.pytorch.org/models/mc3_18-a90a0ba3.pth" to /root/.cache/torch/hub/checkpoints/mc3_18-a90a0ba3.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 90.2MB/s]


400
['abseiling', 'air drumming', 'answering questions', 'applauding', 'applying cream', 'archery', 'arm wrestling', 'arranging flowers', 'assembling computer', 'auctioning']
Топ-3 жанра с вероятностями:
slapping: 0.24
kissing: 0.20
taking a shower: 0.18
